# BigQuery Operations for LLM Experiment Results

This notebook demonstrates how to use the BigQuery client to create datasets, tables, and manage experiment results.

## Setup and Configuration

In [ ]:
import sys
import logging
import pandas as pd
from datetime import datetime
from google.cloud import bigquery
import matplotlib.pyplot as plt
import seaborn as sns

# Add src directory to path
sys.path.append('../src')

from clients.bigquery_client import BigQueryClient

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("BigQuery Operations notebook initialised successfully!")

## Initialise BigQuery Client

Set up the BigQuery client with your project configuration.

In [ ]:
# Configuration - Update these values for your project
PROJECT_ID = "gcp-project-id"
LOCATION = "europe-west2"
DATASET_NAME = "mes-llm_experiments"
TABLE_NAME = "experiment_results"

# Initialise BigQuery client
bq_client = BigQueryClient(
    project_id=PROJECT_ID,
    location=LOCATION
)

print(f"BigQuery client initialised for project: {PROJECT_ID}")
print(f"Location: {LOCATION}")

## Create Dataset and Table

Create the dataset and table structure for storing experiment results.

In [ ]:
from google.cloud import bigquery

def create_experiment_results_table(bq_client, dataset_name: str, table_name: str):
    """
    Create a BigQuery table whose schema mirrors the LLM-evaluation
    results DataFrame.  Field names with illegal characters (e.g. “/”)
    are normalised -> '_' because BigQuery allows only letters,
    numbers and underscores.

    The table is partitioned by DATE(timestamp) and clustered on a few
    high-cardinality dimensions that are handy for filtering.
    """
    # --- 1.  Raw column list taken from the screenshot ----------
    raw_cols = [
        "experiment_name", "model_id", "use_case", "audio_file", "prompt_id", "prompt",
        "response", "reference", "metadata", "experiment_metrics",
        "input_tokens", "output_tokens", "total_tokens", "processing_time",
        "temperature", "top_k", "top_p",
        "timestamp",
        "summarisation_avg_log_probability", "summarisation_confidence",
        "summarisation_section_coverage", "summarisation_required_sections",
        "summarisation_section_length_compliance", "summarisation_format_empty",
        "vertexai_groundedness/explanation", "vertexai_groundedness", "vertexai_summarizationquality/explanation",
        "vertexai_summarizationquality", "vertexai_coherence/explanation",
        "vertexai_coherence", "vertexai_safety/explanation", "vertexai_safety",
        "vertexai_instructionfollowing", "vertexai_instructionfollowing/explanation",
        "vertexai_fluency", "vertexai_fluency/explanation",
        "vertexai_verbosity", "vertexai_verbosity/explanation",
    ]

    # --- 2.  Map every column to a BigQuery type ----------------
    # • Feel free to adjust if you know a column is INT64 vs FLOAT, etc.
    column_type_map = {
        # strings
        "STRING": {
            "experiment_name", "model_id", "use_case", "audio_file", "prompt_id", "prompt",
            "response", "reference", "metadata", "experiment_metrics",
            "vertexai_groundedness/explanation", "vertexai_summarizationquality/explanation", "vertexai_coherence/explanation",
            "vertexai_safety/explanation", "vertexai_instructionfollowing/explanation",
            "vertexai_fluency/explanation", "vertexai_verbosity/explanation",
        },
        # integers
        "INT64": {
            "input_tokens", "output_tokens", "total_tokens", "top_k",
        },
        # floats
        "FLOAT64": {
            "processing_time", "temperature", "top_p",
            "summarisation_avg_log_probability",
            "summarisation_confidence", "summarisation_section_coverage",
            "summarisation_required_sections",
            "summarisation_section_length_compliance",
            "summarisation_format_empty",
            "vertexai_groundedness", "vertexai_summarizationquality",
            "vertexai_coherence", "vertexai_safety",
            "vertexai_instructionfollowing", "vertexai_fluency",
            "vertexai_verbosity",
        },
        # timestamps
        "TIMESTAMP": {"timestamp"},
    }

    schema: list[bigquery.SchemaField] = []
    for col in raw_cols:
        # BigQuery field names can’t contain “/”
        safe_col = col.replace("/", "_")
        # figure out the type
        bq_type = next(typ for typ, cols in column_type_map.items() if col in cols)
        schema.append(bigquery.SchemaField(safe_col, bq_type, mode="NULLABLE"))

    # --- 3.  Create table with partitioning + clustering ----------
    time_partitioning = bigquery.TimePartitioning(
        type_=bigquery.TimePartitioningType.DAY,
        field="timestamp",       # partition on DATE(timestamp)
    )
    clustering_fields = ["experiment_name", "model_id", "use_case"]

    bq_client.create_table(
        dataset_name=dataset_name,
        table_name=table_name,
        schema=schema,
        time_partitioning=time_partitioning,
        clustering_fields=clustering_fields,
        exists_ok=True,
    )


## Insert Sample Data

Create and insert sample experiment data to test the BigQuery integration.

In [ ]:
# Create sample experiment data
sample_data = {
    'experiment_name': ['transcription_baseline', 'transcription_flash', 'transcription_baseline'],
    'model_id': ['gemini-1.5-pro', 'gemini-1.5-flash', 'gemini-1.5-pro'],
    'use_case': ['transcription', 'transcription', 'transcription'],
    'audio_file': ['gs://bucket/audio1.wav', 'gs://bucket/audio2.wav', 'gs://bucket/audio3.wav'],
    'prompt_id': ['transcription-prompt-v1', 'transcription-prompt-v1', 'transcription-prompt-v2'],
    'response_text': ['Sample transcription 1', 'Sample transcription 2', 'Sample transcription 3'],
    'metadata': ['{"confidence": 0.95}', '{"confidence": 0.87}', '{"confidence": 0.92}'],
    'input_tokens': [150, 145, 148],
    'output_tokens': [75, 82, 78],
    'total_tokens': [225, 227, 226],
    'processing_time': [2.34, 1.89, 2.12],
    'temperature': [0.2, 0.2, 0.3],
    'top_k': [40, 40, 32],
    'top_p': [0.8, 0.8, 0.9],
    'timestamp': [datetime.now(), datetime.now(), datetime.now()],
    'run_timestamp': ['20241201_143022', '20241201_143022', '20241201_143022'],
    'inserted_at': [datetime.now(), datetime.now(), datetime.now()],
    'transcript_confidence': [0.95, 0.87, 0.92],
    'transcript_format_compliance': [1.0, 0.95, 1.0],
    'safety_overall': [0.15, 0.12, 0.18],
    'safety_toxicity': [0.05, 0.03, 0.07]
}

sample_df = pd.DataFrame(sample_data)
print(f"Created sample data with {len(sample_df)} rows")
display(sample_df.head())

In [ ]:
# Insert sample data into BigQuery
try:
    bq_client.insert_rows_from_dataframe(
        dataset_name=DATASET_NAME,
        table_name=TABLE_NAME,
        df=sample_df,
        ignore_unknown_values=True,
        skip_invalid_rows=False
    )
    print(f"Successfully inserted {len(sample_df)} rows into BigQuery")
except Exception as e:
    print(f"Error inserting data: {e}")

## Query Experiment Results

Demonstrate querying experiment results from BigQuery.

In [ ]:
# Query all experiment results
query = f"""
SELECT 
    experiment_name,
    model_id,
    prompt_id,
    temperature,
    top_k,
    top_p,
    COUNT(*) as total_runs,
    AVG(transcript_confidence) as avg_confidence,
    AVG(safety_overall) as avg_safety,
    AVG(processing_time) as avg_processing_time,
    SUM(total_tokens) as total_tokens_used
FROM `{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}`
WHERE error IS NULL
GROUP BY experiment_name, model_id, prompt_id, temperature, top_k, top_p
ORDER BY avg_confidence DESC
"""

print("Querying experiment summary...")
results = bq_client.query(query)

# Convert to DataFrame for analysis
query_df = pd.DataFrame([dict(row) for row in results])
print(f"Query returned {len(query_df)} rows")
display(query_df)

In [ ]:
# Query recent experiments
recent_query = f"""
SELECT 
    experiment_name,
    model_id,
    audio_file,
    transcript_confidence,
    safety_overall,
    processing_time,
    timestamp
FROM `{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}`
WHERE inserted_at >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 DAY)
ORDER BY timestamp DESC
LIMIT 10
"""

print("Querying recent experiments...")
recent_results = bq_client.query(recent_query)
recent_df = pd.DataFrame([dict(row) for row in recent_results])

if len(recent_df) > 0:
    print(f"Found {len(recent_df)} recent experiments")
    display(recent_df)
else:
    print("No recent experiments found")

## Visualise BigQuery Results

Create visualisations from BigQuery data.

In [ ]:
# Visualise model performance comparison
if len(query_df) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Confidence by model
    sns.barplot(data=query_df, x='model_id', y='avg_confidence', ax=axes[0,0])
    axes[0,0].set_title('Average Transcript Confidence by Model')
    axes[0,0].tick_params(axis='x', rotation=45)
    
    # Safety by model
    sns.barplot(data=query_df, x='model_id', y='avg_safety', ax=axes[0,1])
    axes[0,1].set_title('Average Safety Score by Model')
    axes[0,1].tick_params(axis='x', rotation=45)
    
    # Processing time by model
    sns.barplot(data=query_df, x='model_id', y='avg_processing_time', ax=axes[1,0])
    axes[1,0].set_title('Average Processing Time by Model')
    axes[1,0].tick_params(axis='x', rotation=45)
    
    # Token usage by model
    sns.barplot(data=query_df, x='model_id', y='total_tokens_used', ax=axes[1,1])
    axes[1,1].set_title('Total Token Usage by Model')
    axes[1,1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
else:
    print("No data available for visualization")

## Advanced BigQuery Analytics

Perform more complex analytics on experiment data.

In [ ]:
# Advanced analytics query
analytics_query = f"""
WITH experiment_stats AS (
  SELECT 
    experiment_name,
    model_id,
    AVG(transcript_confidence) as avg_confidence,
    STDDEV(transcript_confidence) as std_confidence,
    AVG(safety_overall) as avg_safety,
    AVG(processing_time) as avg_processing_time,
    COUNT(*) as sample_count,
    MIN(timestamp) as first_run,
    MAX(timestamp) as last_run
  FROM `{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}`
  WHERE error IS NULL
  GROUP BY experiment_name, model_id
)
SELECT 
  *,
  DATETIME_DIFF(last_run, first_run, MINUTE) as duration_minutes,
  sample_count / DATETIME_DIFF(last_run, first_run, MINUTE) as runs_per_minute
FROM experiment_stats
ORDER BY avg_confidence DESC
"""

print("Running advanced analytics query...")
analytics_results = bq_client.query(analytics_query)
analytics_df = pd.DataFrame([dict(row) for row in analytics_results])

if len(analytics_df) > 0:
    print("Advanced Analytics Results:")
    display(analytics_df)
    
    # Confidence distribution analysis
    plt.figure(figsize=(12, 6))
    for idx, row in analytics_df.iterrows():
        plt.errorbar(
            x=idx, 
            y=row['avg_confidence'], 
            yerr=row['std_confidence'],
            label=f"{row['experiment_name']} ({row['model_id']})",
            capsize=5
        )
    
    plt.xlabel('Experiment')
    plt.ylabel('Transcript Confidence')
    plt.title('Transcript Confidence with Standard Deviation')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()
else:
    print("No data for advanced analytics")

## Prompt and Generation Config Analysis with BigQuery

Demonstrate how to analyze prompt versions and generation configuration impact using BigQuery.

In [ ]:
# Prompt performance comparison query
prompt_analysis_query = f"""
SELECT 
    prompt_id,
    COUNT(*) as total_experiments,
    COUNT(DISTINCT experiment_name) as unique_experiments,
    AVG(transcript_confidence) as avg_confidence,
    AVG(safety_overall) as avg_safety,
    AVG(processing_time) as avg_processing_time,
    STDDEV(transcript_confidence) as confidence_std
FROM `{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}`
WHERE error IS NULL AND prompt_id IS NOT NULL
GROUP BY prompt_id
ORDER BY avg_confidence DESC
"""

print("Analyzing prompt performance...")
try:
    prompt_results = bq_client.query(prompt_analysis_query)
    prompt_df = pd.DataFrame([dict(row) for row in prompt_results])
    
    if len(prompt_df) > 0:
        print("\nPrompt Performance Analysis:")
        display(prompt_df.round(3))
        
        # Visualization
        if len(prompt_df) > 1:
            fig, axes = plt.subplots(1, 2, figsize=(15, 5))
            
            # Confidence comparison
            sns.barplot(data=prompt_df, x='prompt_id', y='avg_confidence', ax=axes[0])
            axes[0].set_title('Average Confidence by Prompt Version')
            axes[0].tick_params(axis='x', rotation=45)
            
            # Processing time comparison
            sns.barplot(data=prompt_df, x='prompt_id', y='avg_processing_time', ax=axes[1])
            axes[1].set_title('Average Processing Time by Prompt Version')
            axes[1].tick_params(axis='x', rotation=45)
            
            plt.tight_layout()
            plt.show()
        else:
            print("Only one prompt version found - no comparison possible")
    else:
        print("No prompt data available for analysis")
        
except Exception as e:
    print(f"Error running prompt analysis: {e}")

In [ ]:
# Generation configuration impact analysis
config_analysis_query = f"""
SELECT 
    temperature,
    top_k,
    top_p,
    COUNT(*) as total_runs,
    AVG(transcript_confidence) as avg_confidence,
    AVG(safety_overall) as avg_safety,
    AVG(processing_time) as avg_processing_time,
    AVG(total_tokens) as avg_tokens
FROM `{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}`
WHERE error IS NULL 
  AND temperature IS NOT NULL 
  AND top_k IS NOT NULL 
  AND top_p IS NOT NULL
GROUP BY temperature, top_k, top_p
ORDER BY avg_confidence DESC
"""

print("Analyzing generation configuration impact...")
try:
    config_results = bq_client.query(config_analysis_query)
    config_df = pd.DataFrame([dict(row) for row in config_results])
    
    if len(config_df) > 0:
        print("\nGeneration Configuration Analysis:")
        display(config_df.round(3))
        
        # Create a correlation analysis
        if len(config_df) > 1:
            print("\nConfiguration Parameter Correlations with Performance:")
            corr_data = config_df[['temperature', 'top_k', 'top_p', 'avg_confidence', 'avg_processing_time']]
            correlation_matrix = corr_data.corr()
            
            plt.figure(figsize=(10, 8))
            sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
            plt.title('Configuration Parameters vs Performance Correlation')
            plt.tight_layout()
            plt.show()
            
            # Temperature impact visualization
            if len(config_df['temperature'].unique()) > 1:
                plt.figure(figsize=(12, 5))
                
                plt.subplot(1, 2, 1)
                sns.scatterplot(data=config_df, x='temperature', y='avg_confidence', size='total_runs')
                plt.title('Temperature vs Confidence')
                
                plt.subplot(1, 2, 2)
                sns.scatterplot(data=config_df, x='temperature', y='avg_processing_time', size='total_runs')
                plt.title('Temperature vs Processing Time')
                
                plt.tight_layout()
                plt.show()
        else:
            print("Only one configuration found - no comparison possible")
    else:
        print("No generation configuration data available for analysis")
        
except Exception as e:
    print(f"Error running config analysis: {e}")

## Data Export and Cleanup

Export data and demonstrate cleanup operations.

In [ ]:
# Export data to CSV
export_query = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}`
WHERE error IS NULL
ORDER BY timestamp DESC
"""

print("Exporting all experiment data...")
export_results = bq_client.query(export_query)
export_df = pd.DataFrame([dict(row) for row in export_results])

if len(export_df) > 0:
    # Save to CSV
    export_filename = f"experiment_results_export_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    export_df.to_csv(export_filename, index=False)
    print(f"Exported {len(export_df)} rows to {export_filename}")
    
    # Show summary
    print("\nExport Summary:")
    print(f"Total experiments: {export_df['experiment_name'].nunique()}")
    print(f"Total models: {export_df['model_id'].nunique()}")
    print(f"Date range: {export_df['timestamp'].min()} to {export_df['timestamp'].max()}")
else:
    print("No data to export")

In [ ]:
# Cleanup operations (optional)
# Uncomment to actually run cleanup

# Delete old test data (older than 7 days)
cleanup_query = f"""
DELETE FROM `{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}`
WHERE inserted_at < TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 7 DAY)
  AND experiment_name LIKE '%test%'
"""

print("Cleanup query prepared (not executed):")
print(cleanup_query)

# To actually run cleanup:
# print("Running cleanup...")
# cleanup_results = bq_client.query(cleanup_query)
# print(f"Cleanup completed")